In [ ]:
#!/usr/bin/python
# -- coding: utf-8 --

import socket
import threading
import time
import os

import ComputerVision as cv

In [ ]:
ip_address = ''
port_number = 2348

server_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_sock.bind((ip_address, port_number))
print("Server socket open...")

print("Listening...")

clients = 100
server_sock.listen(clients)

def makeDynamicHtml():
    html = '<html><head><meta http-equiv="Content-Type" content="text/html; charset=windows-1252"><link rel="stylesheet" type="text/css" href="style.css"><script src="https://code.jquery.com/jquery.min.js"></script><script src="main.js"></script></head>'
    html += '<body><div><div id="smile"><p class="box-name">smile</p><div class="image-box">'
    
    smiles = os.listdir('./faces/smiles')
    
    for smile in smiles:
        imgTag = '<img src="./faces/smiles/' + smile +'">'
        html += imgTag
        
    html += '</div></div>'
    
    html += '<div id="not-smile"><p class="box-name">not_smile</p><div class="image-box">'
    
    not_smiles = os.listdir('./faces/not_smiles')
    
    for not_smile in not_smiles:
        imgTag = '<img src="./faces/not_smiles/' + not_smile +'">'
        html += imgTag
        
    html += '</div></div></div><div><button id="take-photo">take photo</button></div></body></html>'
    
    f = open('./index.html', 'w')
    f.write(html)
    f.close()
    
def recv_data(client_sock, addr):
    makeDynamicHtml()

    data = client_sock.recv(1024)
    decodeData = data.decode()

    headerData = decodeData.split('\n')[0]

    recvMethod = headerData.split()[0]
    body = headerData.split()[1]

    recvMethod = recvMethod.replace(" ", "")
    body = body.replace(" ","")

    filenames = os.listdir("./")

    if body[0] == "/":
        body = body[1:]

    if recvMethod == "GET":
        if (body in filenames) and (body is not 'favicon.ico'):
            f = open(body, 'rb')
            fileSize = os.stat(body).st_size
            responseHeader = 'HTTP/1.1 200 OK\r\nContent-Length: '+ str(fileSize) + '\n\n'
            content = f.read(1024)
            sendMsg = responseHeader.encode() + content
            client_sock.send(sendMsg)

            cycle = int((fileSize - 1024) / 1024) + 1

            while cycle > 0:
                content = f.read(1024)
                client_sock.send(content)
                cycle -= 1
                
            f.close()
            
        elif (len(body.split("/")) is 3 ) and (body.split("/")[2] != '.ipynb_checkpoints'):
            imgPath = body
            f = open(imgPath, 'rb')
            fileSize = os.stat(imgPath).st_size
            responseHeader = 'HTTP/1.1 200 OK\r\nContent-Length: '+ str(fileSize) + '\n\n'
            content = f.read(1024)

            sendMsg = responseHeader.encode() + content
            client_sock.send(sendMsg)

            cycle = int((fileSize - 1024) / 1024) + 1

            while cycle > 0:
                content = f.read(1024)
                client_sock.send(content)
                cycle -= 1
                
            f.close()
            
        else:
            errorMsg = 'HTTP/1.1 404 Not Found\r\n\n'
            client_sock.send(errorMsg.encode())
    elif recvMethod == "POST":
        if decodeData.split('text=')[1] == 'take-photo':
            response = 'HTTP/1.1 200 OK\r\n\n'
            client_sock.send(response.encode())
            cv.smile_detected()
        else:
            errorMsg = 'HTTP/1.1 404 Not Found\r\n\n'
            client_sock.send(errorMsg.encode())
            

    client_sock.close()

while clients > 0:
    client_sock,addr = server_sock.accept()
    threading.Thread(target = recv_data, args = (client_sock,addr,)).start()
    clients -= 1